##### Copyright 2021 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Poda para inferencia en el dispositivo con XNNPACK

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/model_optimization/guide/pruning/pruning_for_on_device_inference"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver en TensorFlow.org</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/es-419/model_optimization/guide/pruning/pruning_for_on_device_inference.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Ejecutar en Google Colab</a>
</td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/es-419/model_optimization/guide/pruning/pruning_for_on_device_inference.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver código fuente en GitHub</a>
</td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/es-419/model_optimization/guide/pruning/pruning_for_on_device_inference.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Descargar el block de notas</a>
</td>
</table>

Le damos la bienvenida a la guía sobre la poda de pesos de Keras para mejorar la latencia de la inferencia en el dispositivo a mediante [XNNPACK](https://github.com/google/XNNPACK).

En esta guía se muestra el uso de la API `tfmot.sparsity.keras.PruningPolicy` recientemente presentada y demuestra cómo podría usarse para acelerar modelos en su mayoría convolucionales en las CPU modernas con la [inferencia dispersa de XNNPACK](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/lite/delegates/xnnpack/README.md#sparse-inference).

La guía cubre los siguientes pasos del proceso de creación del modelo:

- Construir y entrenar la línea de base densa
- Ajustar el modelo con poda
- Convertir a TFLite
- Prueba comparativa en el dispositivo

La guía no cubre las mejores prácticas para el ajuste con poda. Para obtener información más detallada sobre este tema, consulte nuestra [guía completa](https://www.tensorflow.org/model_optimization/guide/pruning/comprehensive_guide.md).

## Preparación

In [ ]:
! pip install -q tensorflow
! pip install -q tensorflow-model-optimization

In [ ]:
import tempfile

import tensorflow as tf
import numpy as np

from tensorflow import keras
import tensorflow_datasets as tfds
import tensorflow_model_optimization as tfmot

%load_ext tensorboard

## Construir y entrenar el modelo denso

Construimos y entrenamos una red neuronal convolucional (CNN) de referencia simple para tareas de clasificación en el conjunto de datos [CIFAR10](https://www.cs.toronto.edu/~kriz/cifar.html).

In [ ]:
# Load CIFAR10 dataset.
(ds_train, ds_val, ds_test), ds_info = tfds.load(
    'cifar10',
    split=['train[:90%]', 'train[90%:]', 'test'],
    as_supervised=True,
    with_info=True,
)

# Normalize the input image so that each pixel value is between 0 and 1.
def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return tf.image.convert_image_dtype(image, tf.float32), label

# Load the data in batches of 128 images.
batch_size = 128
def prepare_dataset(ds, buffer_size=None):
  ds = ds.map(normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
  ds = ds.cache()
  if buffer_size:
    ds = ds.shuffle(buffer_size)
  ds = ds.batch(batch_size)
  ds = ds.prefetch(tf.data.experimental.AUTOTUNE)
  return ds

ds_train = prepare_dataset(ds_train,
                           buffer_size=ds_info.splits['train'].num_examples)
ds_val = prepare_dataset(ds_val)
ds_test = prepare_dataset(ds_test)

# Build the dense baseline model.
dense_model = keras.Sequential([
    keras.layers.InputLayer(input_shape=(32, 32, 3)),
    keras.layers.ZeroPadding2D(padding=1),
    keras.layers.Conv2D(
        filters=8,
        kernel_size=(3, 3),
        strides=(2, 2),
        padding='valid'),
    keras.layers.BatchNormalization(),
    keras.layers.ReLU(),
    keras.layers.DepthwiseConv2D(kernel_size=(3, 3), padding='same'),
    keras.layers.BatchNormalization(),
    keras.layers.ReLU(),
    keras.layers.Conv2D(filters=16, kernel_size=(1, 1)),
    keras.layers.BatchNormalization(),
    keras.layers.ReLU(),
    keras.layers.ZeroPadding2D(padding=1),
    keras.layers.DepthwiseConv2D(
        kernel_size=(3, 3), strides=(2, 2), padding='valid'),
    keras.layers.BatchNormalization(),
    keras.layers.ReLU(),
    keras.layers.Conv2D(filters=32, kernel_size=(1, 1)),
    keras.layers.BatchNormalization(),
    keras.layers.ReLU(),
    keras.layers.GlobalAveragePooling2D(),
    keras.layers.Flatten(),
    keras.layers.Dense(10)
])

# Compile and train the dense model for 10 epochs.
dense_model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer='adam',
    metrics=['accuracy'])

dense_model.fit(
  ds_train,
  epochs=10,
  validation_data=ds_val)

# Evaluate the dense model.
_, dense_model_accuracy = dense_model.evaluate(ds_test, verbose=0)

## Construir el modelo disperso

Siguiendo las instrucciones de la [guía completa](https://www.tensorflow.org/model_optimization/guide/pruning/comprehensive_guide.md), aplicamos la función `tfmot.sparsity.keras.prune_low_magnitude` con parámetros que apuntan a la aceleración en el dispositivo mediante poda, es decir, la directiva `tfmot.sparsity.keras.PruneForLatencyOnXNNPack`.

In [ ]:
prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude

# Compute end step to finish pruning after after 5 epochs.
end_epoch = 5

num_iterations_per_epoch = len(ds_train)
end_step =  num_iterations_per_epoch * end_epoch

# Define parameters for pruning.
pruning_params = {
      'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.25,
                                                               final_sparsity=0.75,
                                                               begin_step=0,
                                                               end_step=end_step),
      'pruning_policy': tfmot.sparsity.keras.PruneForLatencyOnXNNPack()
}

# Try to apply pruning wrapper with pruning policy parameter.
try:
  model_for_pruning = prune_low_magnitude(dense_model, **pruning_params)
except ValueError as e:
  print(e)

La llamada `prune_low_magnitude` da como resultado `ValueError` con el mensaje `Could not find a GlobalAveragePooling2D layer with keepdims = True in all output branches`. El mensaje indica que el modelo no es compatible con la directiva `tfmot.sparsity.keras.PruneForLatencyOnXNNPack` y la capa `GlobalAveragePooling2D`, específicamente, requiere el parámetro `keepdims = True`. Lo arreglamos y volvemos a aplicar la función `prune_low_magnitude`.

In [ ]:
fixed_dense_model = keras.Sequential([
    keras.layers.InputLayer(input_shape=(32, 32, 3)),
    keras.layers.ZeroPadding2D(padding=1),
    keras.layers.Conv2D(
        filters=8,
        kernel_size=(3, 3),
        strides=(2, 2),
        padding='valid'),
    keras.layers.BatchNormalization(),
    keras.layers.ReLU(),
    keras.layers.DepthwiseConv2D(kernel_size=(3, 3), padding='same'),
    keras.layers.BatchNormalization(),
    keras.layers.ReLU(),
    keras.layers.Conv2D(filters=16, kernel_size=(1, 1)),
    keras.layers.BatchNormalization(),
    keras.layers.ReLU(),
    keras.layers.ZeroPadding2D(padding=1),
    keras.layers.DepthwiseConv2D(
        kernel_size=(3, 3), strides=(2, 2), padding='valid'),
    keras.layers.BatchNormalization(),
    keras.layers.ReLU(),
    keras.layers.Conv2D(filters=32, kernel_size=(1, 1)),
    keras.layers.BatchNormalization(),
    keras.layers.ReLU(),
    keras.layers.GlobalAveragePooling2D(keepdims=True),
    keras.layers.Flatten(),
    keras.layers.Dense(10)
])

# Use the pretrained model for pruning instead of training from scratch.
fixed_dense_model.set_weights(dense_model.get_weights())

# Try to reapply pruning wrapper.
model_for_pruning = prune_low_magnitude(fixed_dense_model, **pruning_params)

La invocación de `prune_low_magnitude` finalizó sin errores, lo que significa que el modelo es totalmente compatible con la directiva `tfmot.sparsity.keras.PruneForLatencyOnXNNPack` y se puede acelerar mediante la [inferencia dispersa de XNNPACK](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/lite/delegates/xnnpack/README.md#sparse-inference).

### Ajustar el modelo disperso

Siguiendo el [ejemplo de poda](https://www.tensorflow.org/model_optimization/guide/pruning/pruning_with_keras.md), ajustamos el modelo disperso con los pesos del modelo denso. Comenzamos a ajustar el modelo con un 25 % de dispersión (el 25 % de los pesos se ponen en cero) y terminamos con un 75 % de dispersión.

In [ ]:
logdir = tempfile.mkdtemp()

callbacks = [
  tfmot.sparsity.keras.UpdatePruningStep(),
  tfmot.sparsity.keras.PruningSummaries(log_dir=logdir),
]

model_for_pruning.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer='adam',
    metrics=['accuracy'])

model_for_pruning.fit(
  ds_train,
  epochs=15,
  validation_data=ds_val,
  callbacks=callbacks)

# Evaluate the dense model.
_, pruned_model_accuracy = model_for_pruning.evaluate(ds_test, verbose=0)

print('Dense model test accuracy:', dense_model_accuracy)
print('Pruned model test accuracy:', pruned_model_accuracy)

Los registros muestran el progreso de la dispersión por capa.

In [ ]:
#docs_infra: no_execute
%tensorboard --logdir={logdir}

Después del ajuste con poda, la precisión de la prueba demuestra una mejora modesta (de 43 % a 44 %) en comparación con el modelo denso. Comparemos la latencia en el dispositivo con la [prueba comparativa de TFLite](https://www.tensorflow.org/lite/performance/measurement).

## Conversión de modelos y prueba comparativa

Para convertir el modelo podado en TFLite, necesitamos reemplazar los contenedores `PruneLowMagnitude` con capas originales mediante la función `strip_pruning`. Además, dado que los pesos del modelo podado (`model_for_pruning`) son en su mayoría ceros, podemos aplicar una optimización `tf.lite.Optimize.EXPERIMENTAL_SPARSITY` para almacenar de manera eficiente el modelo TFLite resultante. Este indicador de optimización no es necesario para el modelo denso.

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(dense_model)
dense_tflite_model = converter.convert()

_, dense_tflite_file = tempfile.mkstemp('.tflite')
with open(dense_tflite_file, 'wb') as f:
  f.write(dense_tflite_model)

model_for_export = tfmot.sparsity.keras.strip_pruning(model_for_pruning)

converter = tf.lite.TFLiteConverter.from_keras_model(model_for_export)
converter.optimizations = [tf.lite.Optimize.EXPERIMENTAL_SPARSITY]
pruned_tflite_model = converter.convert()

_, pruned_tflite_file = tempfile.mkstemp('.tflite')
with open(pruned_tflite_file, 'wb') as f:
  f.write(pruned_tflite_model)

Siguiendo las instrucciones de la [herramienta de prueba comparativa de modelos de TFLite](https://github.com/tensorflow/tensorflow/tree/master/tensorflow/lite/tools/benchmark), creamos la herramienta, la subimos al dispositivo Android junto con los modelos de TFLite densos y podados, y comparamos ambos modelos en el dispositivo.

In [ ]:
! adb shell /data/local/tmp/benchmark_model \
    --graph=/data/local/tmp/dense_model.tflite \
    --use_xnnpack=true \
    --num_runs=100 \
    --num_threads=1

In [ ]:
! adb shell /data/local/tmp/benchmark_model \
    --graph=/data/local/tmp/pruned_model.tflite \
    --use_xnnpack=true \
    --num_runs=100 \
    --num_threads=1

Las pruebas comparativas en Pixel 4 dieron como resultado un tiempo de inferencia promedio de *17 us* para el modelo denso y *12 us* para el modelo podado. Las pruebas comparativas del dispositivo demuestran una clara mejora de **5 us** o **30 %** en la latencia incluso para modelos así de pequeños. Según nuestra experiencia, los modelos más grandes que se basan ​​en [MobileNetV3](https://www.tensorflow.org/api_docs/python/tf/keras/applications/mobilenet_v3) o [EfficientNet-lite](https://github.com/tensorflow/tpu/tree/master/models/official/efficientnet/lite) muestran mejoras de rendimiento similares. La aceleración varía según la contribución relativa de las convoluciones 1x1 al modelo general.


## Conclusión

En este tutorial, le mostramos cómo se pueden crear modelos dispersos para un rendimiento más rápido en el dispositivo con la nueva funcionalidad que presentó TF MOT API y XNNPack. Estos modelos dispersos son más pequeños y más rápidos que sus homólogos densos, y al mismo tiempo logran conservar o incluso superar su calidad.

Le recomendamos que pruebe esta nueva capacidad, que puede ser particularmente importante para implementar sus modelos en el dispositivo.